In [ ]:
import requests
import pandas as pd
import numpy as np
import re

In [ ]:
# цель - анализ финансовых показателей компаний среди разработчиков информационных технологий

# Создание списка компаний

In [ ]:
url1 = 'https://www.testfirm.ru/'

## 1 вариант - Список из первых 50 фирм - с первой страницы фильтра

In [ ]:
url_list = 'https://www.testfirm.ru/rating' # рейтинг по всем предприятиям 
k_1 = 63   #код деятельности
k_2 = "sverdlovskaya-oblast"  #область
url_list_copm = url_list + '/' + str(k_1) + '/' + k_2 + '/'
text = requests.get(url_list_copm) # 
try:
  text.status_code == 200
  print('Список загружен нормально')
except:
  print('ОШИБКА! Проверяй!')

Список загружен нормально


In [ ]:
# поиск названия компании
text_write = text.text

def find_komp(text_write, N):
    """ф. для поиска ИНН и названия по каждой компании - для формирования url-ссылки на информацию по этой компании
    """
    firma_list = text_write.split('org-link" href="/result/')
    return firma_list[N].split('"')[0]

def find_komp_re(text_write, N):
  """  ф. для поиска ИНН и названия по каждой компании - с помощью регулярных выражений
  """
  pass    

## 2 вариант - Список из всех фирм фильтра

### - загрузим страницу с рейтингом компаний

In [ ]:
# Рейтинг организаций по выручке - Вид деятельности: 63 "Деятельность в области информационных технологий"
url_list_all = url1 + 'rating/63/' # адрес страницы с рейтингом
text_list_all = requests.get(url_list_all) # 
try:
  text_list_all.status_code == 200
  print('Список загружен нормально')
except:
  print('ОШИБКА! Проверяй!')

Список загружен нормально


In [ ]:
def find_name(url):
    """ ф. находит инфу для url страницы с информацией по компании - состоит из ИНН и названия Компании
    -  inn_firma 
    """
    text = requests.get(url)
    text_find = text.text 
    text_find = text_find.split('class="org-link"')[1].split('result/')[1].split('">')[0]
    return text_find

def find_name_next(url, koef = 1):
    """ ф. находит инфу для url страницы с информацией по следующей компании
    """
    text = requests.get(url)
    text_find = text.text 
    text_find = text_find.split('class="org-link"')[koef].split('result/')[1].split('">')[0]
    return text_find  

In [ ]:
#url_one = 'https://www.testfirm.ru/result/' + name #   https://www.testfirm.ru/result/7708119944_ooo-sibirskaya-internet-kompaniya

def find_info_comp(url_one):
    """ ф.собирает информацию по каждой компании и сохраняет данные в строку
    """
    stroka = [] # собираем информацию сюда

    text_one = requests.get(url_one)
    text_write = text_one.text
    a1 = text_write.split('Организация: ')[1]
 #   stroka.append(a1.replace(' &quot;', '').split('<')[0].split(' ')[0]) # правовая форма
    if text_write.split('id="searchText" name="s" value=')[1].split('placeholder')[0]: # название
        stroka.append(text_write.split('id="searchText" name="s" value=')[1].split('placeholder')[0])
    else:
        stroka.append(0)
#    print(stroka[-1])
    stroka.append(a1.split('ИНН: ')[1][:10]) # ИНН
#    print(stroka[-1])
    stroka.append(a1.split('<b>')[2].split('<')[0].rstrip()) # активы
#    print(stroka[-1])
    mln = a1.split('Активы на ')[1].split('/b>')[1][:5].rstrip() # млн или тыс
    if mln in 'млн.':
        stroka.append(1000000)
    elif mln in 'тыс.':
        stroka.append(1000)
#    print(stroka[-1])
    stroka.append(a1.split('Выручка')[1].split('b>')[1].split('<')[0].rstrip()) # выручка
#    print(stroka[-1])
    mln = a1.split('Выручка')[1].split('b>')[2].split('руб')[0].rstrip() # млн ии тыс
    if mln in 'млн.':
        stroka.append(1000000)
    elif mln in 'тыс.':
        stroka.append(1000)
#    print(stroka[-1])
    if 'численности ' in text_write:
        stroka.append(a1.split('численности ')[1].split('b>')[1].split('<')[0]) # численность человек
#        print(stroka[-1])
    else:
        stroka.append(0) 
    if 'Дополнительные данные ФНС' in text_write:
        stroka.append(a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[1].split('<')[0].rstrip()) # доходы 
#        print(stroka[-1])
        stroka.append(a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[2].split('<')[0].rstrip()) # расходы
#        print(stroka[-1])
    else:
        stroka.append(0)
        stroka.append(0)
    if "Уплачено налогов" in text_write:
        stroka.append(text_write.split("Уплачено налогов")[1].split('b>')[1].split('<')[0].rstrip()) # налоги уплаченные
    else:
        stroka.append(0)
    return stroka


In [ ]:
# проверка
Table_find_values = [] 

name = find_name(url_list_all)      # ф. найдет определитель компании - inn_firma
url_one = url1 + 'result/' + name   # url страницы с информацией о компании
text_one = requests.get(url_one)    # создат объект Response
a = find_info_comp(url_one)             # ф. покажет инф. о компании
#print(a)
Table_find_values.append(a)
inn = name.split('_')[0]            # выделить инн
url_1 = url1 + 'rating/63_11/' + inn + '/' # url страницы с местом в рейтинге  / вернутся в рейтинг фирм
#print(url_1)
#print('\t \t 1')

name_next_1 = requests.get(url_1).text.split(name)[1].split('href="/result/')[1].split('">')[0] # найти name следующей компании
inn_next = name_next_1.split('_')[0]
url_next_company = url1 + 'result/' + name_next_1
a = find_info_comp(url_next_company) #  ф. покажет инф. о компании
#print(a)
Table_find_values.append(a)
url_1 = url1 + 'rating/63_11/' + inn_next + '/' # url страницы с местом в рейтинге  / вернутся в рейтинг фирм
#print(url_1)
#print('\t \t 2')

for i in range(6000):
    name_next_1 = requests.get(url_1).text.split(name_next_1)[1].split('href="/result/')[1].split('">')[0] # найти name следующей компании
    inn_next = name_next_1.split('_')[0]
    url_next_company = url1 + 'result/' + name_next_1
    a = find_info_comp(url_next_company) #  ф. покажет инф. о компании
 #   print(a)
    Table_find_values.append(a)
    url_1 = url1 + 'rating/63_11/' + inn_next + '/' # url страницы с местом в рейтинге  / вернутся в рейтинг фирм 
 #   print(url_1)
    if (i - 3) % 100 == 0:
        print('\t \t', i + 3)


	 	 6
	 	 106
	 	 206
	 	 306
	 	 406
	 	 506
	 	 606
	 	 706
	 	 806
	 	 906
	 	 1006
	 	 1106
	 	 1206
	 	 1306
	 	 1406
	 	 1506
	 	 1606
	 	 1706
	 	 1806
	 	 1906
	 	 2006
	 	 2106
	 	 2206
	 	 2306
	 	 2406
	 	 2506
	 	 2606
	 	 2706
	 	 2806
	 	 2906
	 	 3006
	 	 3106
	 	 3206
	 	 3306
	 	 3406
	 	 3506
	 	 3606
	 	 3706
	 	 3806
	 	 3906
	 	 4006
	 	 4106
	 	 4206
	 	 4306
	 	 4406
	 	 4506
	 	 4606
	 	 4706
	 	 4806
	 	 4906
	 	 5006
	 	 5106
	 	 5206
	 	 5306
	 	 5406
	 	 5506
	 	 5606
	 	 5706
	 	 5806
	 	 5906


### - найдем информацию для поиска компании

### - определим функцию для поиска информации по каждой компании 

# Сбор данных по каждой компании

In [ ]:
def find_info_comp_re(List_all_komp, n_comp):
    """ ф.собирает информацию по каждой компании и сохраняет данные в строку с помощью регулярных выражений
    """
    pass


# Анализ данных

In [ ]:
data = pd.DataFrame(Table_find_values)
data.columns = ['Наименование', 'ИНН', 'Активы', 'ед.изм.', 'Выручка', 'ед.изм.', 'Численность персонала', 'Доходы', 'Расходы', 'Налоги'] 
data.head()

,Наименование,ИНН,Активы,ед.изм.,Выручка,ед.изм.,Численность персонала,Доходы,Расходы,Налоги
0,"""Сибирская Интернет Компания""",7708119944,47 403,1000000,52 074,1000000,0,0,0,0
1,"""МЭЙЛ РУ""",7743001840,198 742,1000000,45 582,1000000,0,0,0,0
2,"""Центр программ лояльности""",7702770003,8 723,1000000,32 778,1000000,166,33 174,30 617,1 680
3,"""НАЦИОНАЛЬНАЯ СИСТЕМА ПЛАТЕЖНЫХ КАРТ""",7706812159,23 191,1000000,16 171,1000000,0,0,0,0
4,"""Платежная система Виза""",7710759236,3 079,1000000,15 036,1000000,0,0,0,0


In [ ]:
data.shape

(6002, 10)

In [ ]:
data.to_csv('data_find_values.csv')

In [ ]:
data.describe()

,ед.изм.,ед.изм.
count,6002.000000,6002.0
mean,999334.221926,1000000.0
std,25783.323974,0.0
min,1000.000000,1000000.0
25%,1000000.000000,1000000.0
50%,1000000.000000,1000000.0
75%,1000000.000000,1000000.0
max,1000000.000000,1000000.0
